# Система прогнозирования прибыльности и выбора локаций для бурения скважин в компании «ГлавРосГосНефть»

_____
_____
## Описание проекта

Добывающая компания «ГлавРосГосНефть» стремится повысить эффективность добычи нефти, определяя регионы с максимальным потенциалом прибыли. Для этого необходимо разработать систему, которая позволит анализировать данные геологоразведки, прогнозировать объём запасов нефти и принимать решения о бурении скважин в наиболее выгодных точках.

- **Цели и задачи проекта:**

    - **Прогнозирование запасов нефти**: Создание модели машинного обучения, способной точно оценивать объём нефти в скважинах на основе характеристик месторождений. Это поможет отбирать перспективные точки для разработки.  

    - **Оценка прибыли и рисков**: Анализ доходности бурения с учётом бюджета и рыночных условий. Использование техники Bootstrap для расчёта распределения прибыли, определения риска убытков и обоснования решений.  

    - **Оптимизация выбора региона**: На основе модели выбрать регион с наибольшей суммарной прибылью и минимальной вероятностью убытков, соблюдая установленные бизнес-ограничения.  

    - **Ключевые требования к системе**:  
        - Высокая точность прогнозов объёма запасов.  
        - Возможность оценки рисков и прибыли для каждого региона.  
        - Удобная интеграция модели в текущие бизнес-процессы компании.

_____
_____
## Импорт и подготовка к работе

In [ ]:
from copy import deepcopy
from itertools import cycle
from pprint import pprint
from typing import Literal

import numpy as np
import optuna
import pandas as pd
from phik import report
import shap
from bidict import bidict
from category_encoders import (JamesSteinEncoder,
                               LeaveOneOutEncoder,
                               MEstimateEncoder,
                               WOEEncoder)
from optuna.visualization import plot_param_importances
import plotly
from plotly.colors import find_intermediate_color, hex_to_rgb
from plotly.subplots import make_subplots
from plotly import express as px, graph_objects as go
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import RFECV, SelectKBest, f_classif
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (MinMaxScaler,
                                   OneHotEncoder,
                                   OrdinalEncoder,
                                   StandardScaler)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant




from sklearn.preprocessing import RobustScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error

In [ ]:
pd.set_option('display.max_columns', None)
optuna.logging.set_verbosity(optuna.logging.WARNING)
plotly.offline.init_notebook_mode()

RANDOM_STATE = 42
optuna_sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)
color_palette = cycle(px.colors.qualitative.Plotly)

_____
_____
## Загрузка данных и общая информация

_____
### Скачивание датасетов и общая информация

Три таблицы имеет данные о скважинах из трех регионов соответственно и имеет вид: 

|Поле                               |Описание                                |
|-----------------------------------|----------------------------------------|
|id                                 |Уникальный идентификатор скважины       |
|f0                                 |Параметр 1                              |
|f1                                 |Параметр 2                              |
|f2                                 |Параметр 3                              |
|product                            |Объём запасов в скважине (тыс. баррелей)|

1. **`geo_data_0.csv`** (Первый регион)

In [ ]:
try:
    regions_wells_1 = pd.read_csv('geo_data_0.csv', delimiter=',')
except:
    regions_wells_1 = pd.read_csv('/datasets/geo_data_0.csv', delimiter=',')
display(regions_wells_1.head())
regions_wells_1.info()

2. **`geo_data_1.csv`** (Второй регион)

In [ ]:
try:
    regions_wells_2 = pd.read_csv('geo_data_1.csv', delimiter=',')
except:
    regions_wells_2 = pd.read_csv('/datasets/geo_data_1.csv', delimiter=',')
display(regions_wells_2.head())
regions_wells_2.info()

3. **`geo_data_2.csv`** (Третий регион)

In [ ]:
try:
    regions_wells_3 = pd.read_csv('geo_data_2.csv', delimiter=',')
except:
    regions_wells_3 = pd.read_csv('/datasets/geo_data_2.csv', delimiter=',')
display(regions_wells_3.head())
regions_wells_3.info()

_____
### Вывод:

- `regions_wells_1` (`geo_data_0.csv`)
    - Содержит данные о скважинах первого региона.
    - Всего 100,000 строк.
    - Пропущенных значений нет.
    - 3 количественных признака: `f0`, `f1`, `f2`, `product`.
    - 1 категориальный признак: `id`.
    - Типы данных верные.

- `regions_wells_2` (`geo_data_1.csv`)
    - Содержит данные о скважинах первого региона.
    - Всего 100,000 строк.
    - Пропущенных значений нет.
    - 3 количественных признака: `f0`, `f1`, `f2`, `product`.
    - 1 категориальный признак: `id`.
    - Типы данных верные.

- `regions_wells_3` (`geo_data_2.csv`)
    - Содержит данные о скважинах первого региона.
    - Всего 100,000 строк.
    - Пропущенных значений нет.
    - 3 количественных признака: `f0`, `f1`, `f2`, `product`.
    - 1 категориальный признак: `id`.
    - Типы данных верные.

_____
_____
## Предобработка данных

_____
### Наименовывание столбцов датафреймов

Все столбцы названы корректно, однако для удобства переименуем таргет.

In [6]:
regions_wells_1 = regions_wells_1.rename(columns={'product': 'well_reserves_volume'})
regions_wells_2 = regions_wells_2.rename(columns={'product': 'well_reserves_volume'})
regions_wells_3 = regions_wells_3.rename(columns={'product': 'well_reserves_volume'})

columns_translate = bidict({
    'id': 'id',
    'f0': 'Признак 1',
    'f1': 'Признак 2',
    'f2': 'Признак 3',
    'well_reserves_volume': 'Объем запасов в скважине',
})

_____
### Обработка дубликатов и опечаток

- ***Явные***

In [ ]:
print('Кол-во явных дубликатов в regions_wells_1:\t', regions_wells_1.id.duplicated().sum())
print('Кол-во явных дубликатов в regions_wells_2:\t', regions_wells_2.id.duplicated().sum())
print('Кол-во явных дубликатов в regions_wells_3:\t', regions_wells_3.id.duplicated().sum())

In [8]:
regions_wells_1 = regions_wells_1.drop_duplicates(subset=['id'])
regions_wells_2 = regions_wells_2.drop_duplicates(subset=['id'])
regions_wells_3 = regions_wells_3.drop_duplicates(subset=['id'])

Явные дубликаты удалены

- ***Неявные***

In [ ]:
print('Кол-во явных дубликатов в regions_wells_1:\t', regions_wells_1[['f0', 'f1', 'f2', 'well_reserves_volume']].duplicated().sum())
print('Кол-во явных дубликатов в regions_wells_2:\t', regions_wells_2[['f0', 'f1', 'f2', 'well_reserves_volume']].duplicated().sum())
print('Кол-во явных дубликатов в regions_wells_3:\t', regions_wells_3[['f0', 'f1', 'f2', 'well_reserves_volume']].duplicated().sum())

Неявные дубликаты отсутствуют

_____
### Обработка аномальных значений и выбросов

In [10]:
def plot_boxplots(data: pd.DataFrame, data_name: str, row: int, cols: int, height: int, weight: int):
    fig = make_subplots(rows=row, cols=cols, vertical_spacing=0.07,
                        subplot_titles=[columns_translate[name]
                                        for name in data.select_dtypes(include=['number']).columns])
    for i, column in enumerate(data.select_dtypes(include=['number']).columns):
        fig.add_trace(
            go.Box(y=data[column], name=''),
            row=(i // cols) + 1, col=(i % cols) + 1
        )
    fig.update_layout(height=height, width=weight, showlegend=False, title=f'График "Ящик с усами" для каждой количественной фичи в {data_name}')
    fig.show()

In [ ]:
plot_boxplots(regions_wells_1, 'regions_wells_1', row=1, cols=4, height=600, weight=1200)
plot_boxplots(regions_wells_2, 'regions_wells_2', row=1, cols=4, height=600, weight=1200)
plot_boxplots(regions_wells_3, 'regions_wells_3', row=1, cols=4, height=600, weight=1200)

- Математически есть выбросы, однако нельзя их убирать, так как:
    1. Они не сильно выбиваются;
    2. Из-за незнания что эти столбцы хранят;
    3. Может эти выбивающиеся значения и есть скважины с большим кол-вом нефти.

_____
### Смена индекса датафреймов

In [12]:
regions_wells_1 = regions_wells_1.set_index('id')
regions_wells_2 = regions_wells_2.set_index('id')
regions_wells_3 = regions_wells_3.set_index('id')

_____
### Вывод:

- Удалил неявные дубликаты.
- Обработал аномальные значения и выбросы.
- Поменял индексы датафреймов.

_____
_____
## Анализ данных

_____
### Функции отрисовок данных

In [13]:
def statistical_graphis_for_numeric(col: pd.Series, title_text=None, nbinsx=50):
    """
    Функция для построения графиков для числовых данных: гистограммы и диаграммы размаха.
    
    Parameters
    ----------
    col : pd.Series
        Входные данные в виде столбца (серии) pandas, представляющие числовую переменную.
        
    nbinsx : int, по умолчанию 50
        Количество корзин (bins) для построения гистограммы. Управляет точностью распределения данных по оси x.
    """
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Гистограмма', 'Диаграмма размаха'))
    
    fig.add_trace(
        go.Histogram(x=col, nbinsx=nbinsx, marker_color='blue', name=col.name),
        row=1, col=1
    )
    fig.update_xaxes(title_text=columns_translate[col.name], row=1, col=1)
    fig.update_yaxes(title_text='Частота', row=1, col=1)

    fig.add_trace(
        go.Box(y=col, marker_color='orange', name=''),
        row=1, col=2
    )
    fig.update_yaxes(title_text=columns_translate[col.name], row=1, col=2)

    fig.update_layout(
        title_text=(title_text if title_text
                    else f'Статистические графики по значению <b>{columns_translate[col.name]}</b> (<b>{col.name}</b>)'),
        title_x=0.5,
        showlegend=False,
        width=1200,
        height=500
    )

    fig.show()

_____
### `regions_wells_1`

In [ ]:
for num_col in regions_wells_1.select_dtypes(include=['number']).columns:
    statistical_graphis_for_numeric(
        col=regions_wells_1[num_col],
        nbinsx={'f0': 50, 'f1': 50, 'f2': 50, 'well_reserves_volume': 80}[num_col]
    )

_____
### `regions_wells_2`

In [ ]:
for num_col in regions_wells_2.select_dtypes(include=['number']).columns:
    statistical_graphis_for_numeric(
        col=regions_wells_2[num_col],
        nbinsx={'f0': 50, 'f1': 50, 'f2': 20, 'well_reserves_volume': 20}[num_col]
    )

_____
### `regions_wells_3`

In [ ]:
for num_col in regions_wells_3.select_dtypes(include=['number']).columns:
    statistical_graphis_for_numeric(
        col=regions_wells_3[num_col],
        nbinsx={'f0': 50, 'f1': 50, 'f2': 50, 'well_reserves_volume': 80}[num_col]
    )

_____
### Вывод:

- Данные многомодально распределены.
- Некоторые признаки (причем в зависимости от скважин) дискретны.
- Обязательно стоит рассмотреть корреляцию между признаками, потому что некоторые признаки как будто дополняют друг друга (к примеру признаки 1 и 2 из первой скважины).

_____
_____
## Корреляционный анализ

In [17]:
def self_binned_df(data: pd.DataFrame, bins=100):
    binned_data = data.copy()
    for num_col in data.select_dtypes(include=['number']).columns:
        binned_data[num_col] = pd.cut(binned_data[num_col], bins, labels=False)
    return binned_data


def corr_and_significance_matrices(data: pd.DataFrame, data_name: str, interval_cols: list[str] = None):
    display(px.imshow(
        data.rename(
            columns=columns_translate
        ).phik_matrix(interval_cols), 
        text_auto='.2f',
        color_continuous_scale='oranges',
        title=f'Матрица корреляции для <b>{data_name}</b>'
    ).update_layout(
        width=800,
        height=800,
        title_font_size=20,
        font=dict(size=14)
    ))
    display(px.imshow(
        data.rename(
            columns=columns_translate
        ).significance_matrix(interval_cols),
        text_auto='.2f',
        color_continuous_scale='oranges',
        title=f'Матрица значимости для <b>{data_name}</b>'
    ).update_layout(
        width=800,
        height=800,
        title_font_size=20,
        font=dict(size=14)
    ))

_____
### `regions_wells_1`

In [ ]:
corr_and_significance_matrices(self_binned_df(regions_wells_1), 'regions_wells_1')

_____
### `regions_wells_2`

In [ ]:
corr_and_significance_matrices(self_binned_df(regions_wells_2), 'regions_wells_2')

_____
### `regions_wells_3`

In [ ]:
corr_and_significance_matrices(self_binned_df(regions_wells_3), 'regions_wells_3')

|Корреляция с `Объем запасов в скважине`|Первый регион|Второй регион|Третий регион|
|---------------------------------------|-------------|-------------|-------------|
|Признак 1                              |слабая       |умеренная    |слабая       |
|Признак 2                              |слабая       |слабая       |слабая       |
|Признак 3                              |умеренная    |сильная      |умеренная    |

- Матрица значимости во всех случаех показывает аналогичные результаты.
- Стоит проверить признаки на мультиколлинеарность

*Проверка на мультиколлинеарность через VIF*

$VIF = \frac{1}{1-R^2}$
- Если $VIF ≈ 1$: мультиколлинеарности нет.
- Если  $5 ≤ VIF < 10$: возможна умеренная мультиколлинеарность.
- Если  $VIF ≥ 10$: сильная мультиколлинеарность.

In [ ]:
vif = pd.DataFrame()
for i in range(3):
    regions_wells = [regions_wells_1, regions_wells_2, regions_wells_3][i]
    X = add_constant(regions_wells.drop('well_reserves_volume', axis=1))
    vif[f'regions_wells_{i}'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
else:
    vif['features'] = X.columns
    vif = vif.set_index('features')
vif

- Мультиколлинеарность между признаками отсутствует

_____
### Вывод:

- **Корреляция с целевым признаком**:
    - `Признак 1` умеет умеренную корреляцию во втором регионе и слабую в остальных.
    - `Признак 2` слабо коррелирует во всех регионах.
    - `Признак 3` стабильно показывает среднее-высокое значение во всех регионах.

- **Мультиколлинеарность**:
    - Среди признаков не наблюдается мультиколлинеарность.

_____
_____
## Модели

_____
### Оценка моделей

In [22]:
def feature_selection_perfomance(*results, model_names: list[str]):
    """
    Функция для визуализации результатов отбора признаков с использованием mse или средней точности теста для различных моделей.
    Создает график, показывающий зависимость mse (или точности) моделей от количества выбранных признаков.

    Parameters
    ----------
    *results : dict
        - `dict`, содержащий результаты кросс-валидации модели, включая ключ 'mean_test_score' для средней точности модели.

    model_names : list of str
        Список с именами моделей, которые соответствуют каждому из результатов.
    """
    fig = go.Figure()
    for i, result in enumerate(results, start=1):
        num_features = np.arange(1, len(result['mean_test_score']) + 1)
        mean_test_scores = result['mean_test_score']
        fig.add_trace(go.Scatter(x=num_features,
                                    y=mean_test_scores,
                                    mode='lines+markers',
                                    name=model_names[i-1],
                                    legendgroup=str(i),
                                    legendgrouptitle=dict(text=f'{model_names[i-1]}')))

        max_index = np.argmax(mean_test_scores)
        fig.add_trace(go.Scatter(x=[num_features[max_index]],
                                    y=[mean_test_scores[max_index]],
                                    mode='markers',
                                    marker=dict(color='red', size=10),
                                    showlegend=False,
                                    name=model_names[i-1],
                                    legendgroup=str(i),
                                    legendgrouptitle=dict(text=f'{model_names[i-1]}')))
    fig.update_layout(
        title_text='MSE моделей в зависимости от числа признаков',
        xaxis_title='Число признаков',
        yaxis_title='mse',
        showlegend=True
    )
    fig.show()

_____
### <a id='toc1_8_2_'></a>[Пайплайн](#toc0_)

In [23]:

def get_pipepline_reg(numerical_columns: list[str]) -> Pipeline:
    """
    Функция для создания пайплайна регрессии с различными методами предварительной обработки данных и отбором признаков.
    
    Parameters
    ----------

    numerical_columns : list of str
        Список названий числовых признаков.
    
    Returns
    -------
    Pipeline
        Возвращает объект `Pipeline`, который включает в себя этапы предварительной обработки и этап отбора признаков.
    """
    numerical_preprocessor = Pipeline(
        steps=[('imputer', 'passthrough'),
               ('scaler', 'passthrough')]
    )
    preprocessor = ColumnTransformer(
        [('numerical', numerical_preprocessor, numerical_columns)],
        verbose_feature_names_out=False,
    )
    model = RFECV(estimator=LinearRegression(n_jobs=2),
                  min_features_to_select=1,
                  step=1,
                  scoring='neg_mean_squared_error',
                  n_jobs=5,
                  cv=5,
                  verbose=0)
    return Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ]
    )

*Датасеты*

In [24]:
X1 = regions_wells_1.drop('well_reserves_volume', axis=1)
y1 = regions_wells_1.well_reserves_volume
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X1, y1, random_state=RANDOM_STATE)

In [25]:
X2 = regions_wells_2.drop('well_reserves_volume', axis=1)
y2 = regions_wells_2.well_reserves_volume
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X2, y2, random_state=RANDOM_STATE)

In [26]:
X3 = regions_wells_3.drop('well_reserves_volume', axis=1)
y3 = regions_wells_3.well_reserves_volume
X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(X3, y3, random_state=RANDOM_STATE)

_____
### <a id='toc1_8_3_'></a>[Перебор гиперпараметров через optuna](#toc0_)

*Определяю перебираемые параметры для препроцессинга*

In [27]:
preprocessor_params = {
    'preprocessor__numerical__scaler': [
        MinMaxScaler(),
        RobustScaler(),
        MaxAbsScaler(),
        QuantileTransformer(output_distribution='normal', random_state=RANDOM_STATE),
        QuantileTransformer(output_distribution='uniform', random_state=RANDOM_STATE),
        PowerTransformer(method='yeo-johnson', standardize=True),
        StandardScaler(),
    ],
    'preprocessor__numerical__imputer': [
        SimpleImputer(strategy='median'),
        KNNImputer(n_neighbors=5),
        IterativeImputer(estimator=DecisionTreeRegressor(random_state=RANDOM_STATE)),
        IterativeImputer(estimator=BayesianRidge()),
        IterativeImputer(max_iter=20, random_state=RANDOM_STATE),
        SimpleImputer(strategy='mean'),
    ]
}

In [28]:
X_train, y_train, X_val, y_val = None, None, None, None
pipeline = None

*Определяю функции для перебора гиперпараметров*

In [29]:
def objective(trial : optuna.trial.Trial) -> float:
    """
    Целевая функция для оптимизации гиперпараметров с использованием библиотеки Optuna.
    В этой функции выполняется настройка и обучение модели с использованием предложенных значений гиперпараметров,
    а затем оценивается её точность на валидационной выборке.

    Parameters
    ----------
    trial : optuna.trial.Trial
        Экземпляр объекта `Trial` из библиотеки Optuna, который используется для выбора гиперпараметров.

    Returns
    -------
    float
        Оценка модели (например, точность) на валидационной выборке. Если возникает ошибка, пробный эксперимент прерывается.
    """
    params = {
        param_name: preprocessor_params[param_name][trial.suggest_categorical(param_name, range(0, len(preprocessor_params[param_name])))]
        for param_name in preprocessor_params
    }
    pipeline_temp:Pipeline = clone(pipeline)
    pipeline_temp.set_params(**params)
    try:
        pipeline_temp.fit(X_train, y_train)
        return mean_squared_error(y_val, pipeline_temp.predict(X_val), squared=False)
    except ValueError as e: # Обработка несовместных параметров
        print(e)
        raise optuna.exceptions.TrialPruned()


def get_best_params(best_params):
    return {
        param: preprocessor_params[param][best_params[param]] if param in preprocessor_params
                                                              else best_params[param]
        for param in best_params
    }

_____
### Первый регион

In [30]:
X_train, y_train, X_val, y_val = X_train_1, y_train_1, X_val_1, y_val_1

pipeline = get_pipepline_reg(X_train.select_dtypes(include='number').columns)

reg_region_1 = optuna.create_study(direction='minimize', sampler=optuna_sampler)

In [ ]:
%%time
reg_region_1.optimize(objective, n_trials=300, show_progress_bar=True)

In [ ]:
print(f'Best precision: {reg_region_1.best_value:.3f}')
best_params = get_best_params(reg_region_1.best_params)
print('\nBest parameters:')
pprint(best_params)

*Важность признаков судя по перебору*

In [ ]:
plot_param_importances(reg_region_1)

- Так как в валидационных данных не было пропусков, то и значимость признака оценить нельзя, поэтому если модель будет далее использоваться, то необходимо сделать искусственные пропуски в валидационном датасете и переобучить.

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_4_2_'></a>[Feature Engineering](#toc0_)

*Обучим модель на лучших параметрах*

In [ ]:
reg_region_1 = clone(pipeline).set_params(**best_params)
reg_region_1.fit(X_train, y_train)
reg_region_1

In [ ]:
mean_squared_error(y_val, reg_region_1.predict(X_val), squared=False)

*График зависимости качества модели от количества признаков*

In [ ]:
feature_selection_perfomance(reg_region_1.named_steps['model'].cv_results_,
                             model_names=['Регион 1'])

- Модель показывает хорошие результаты, ROC AUC топ конфигурации является 0.899
- Лучшее кол-во признаков это 8
- Модель достаточно стабильна относительно кол-ва признаков

*Интерпретация важности признаков через shap*

In [ ]:
X_val_transformed = reg_region_1.transform(X_val)
final_feature_names = np.array(reg_region_1.get_feature_names_out())
explainer = shap.LinearExplainer(reg_region_1.named_steps['model'].estimator_,
                                 shap.maskers.Independent(X_val_transformed), 
                                 feature_names=final_feature_names)
shap_values = explainer.shap_values(X_val_transformed)
shap.summary_plot(shap_values,
                  X_val_transformed,
                  feature_names=final_feature_names,
                  plot_type='violin')

- На изображении можно наблюдать все признаки, отобранные RFECV, и их качество по мнению shap
- Важными для логистической регресии являются признаки `avg_categories_per_visit`, `pages_per_visit` и `spent_time_prev_month`
- При этом эти признаки обратнопропопрциональны таргету. Также их распределение равномерно и не близко к 0, что показывает их большое качество.
- Прямопропорционально таргету являются признаки `promotional_purchases` и `unpaid_products_quarter`, что вполне логично.

_____
### <a id='toc1_8_5_'></a>[Дерево решений](#toc0_)

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_5_1_'></a>[Подбор гиперпараметров](#toc0_)

In [ ]:
pipeline = get_pipepline_clf(
    feature_selection='RFECV',
    numerical_columns=market.select_dtypes(include='number').columns,
    categorical_columns=['popular_category'],
    categorical_ordered_columns=['service_type', 'allow_notifications']
)

model_params_cat = {'model__estimator': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
                    'model__estimator__criterion': ['gini', 'entropy', 'log_loss'],
                    'model__estimator__splitter': ['best', 'random']}
model_params_num = {
    'model__estimator__max_depth': lambda trial: trial.suggest_int('model__estimator__max_depth', 3, 40),
    'model__estimator__min_samples_split': lambda trial: trial.suggest_int('model__estimator__min_samples_split', 2, 20),
    'model__estimator__min_samples_leaf': lambda trial: trial.suggest_int('model__estimator__min_samples_leaf', 1, 10)
}

study_tree = optuna.create_study(direction='maximize')

In [ ]:
%%time
study_tree.optimize(objective, n_trials=300, show_progress_bar=True)

In [ ]:
print(f'Best ROC AUC: {study_tree.best_value:.3f}')
best_params = get_best_params(study_tree.best_params)
print('\nBest parameters:')
pprint(best_params)

*Важность признаков судя по перебору*

In [ ]:
plot_param_importances(study_tree)

- Самым влиятельным для модели гиперпараметром является `preprocessor__categorical_ordered__encoder`
- Менее влиятельными, но тоже важными можно считать `preprocessor__catatigorical__encoder` и `model__estimator__splitter`

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_5_2_'></a>[Feature Engineering](#toc0_)

*Обучим модель на лучших параметрах*

In [ ]:
rfecv_tree = clone(pipeline).set_params(**best_params)
rfecv_tree.fit(X_train, y_train.cat.codes)
rfecv_tree

*График зависимости качества модели от количества признаков*

In [ ]:
feature_selection_perfomance(rfecv_tree.named_steps['model'].cv_results_,
                             model_names=['Дерево решений'])

In [ ]:
rfecv_tree.get_feature_names_out()

- RFECV по итогу отобрал всего 5 признаков
- Качество на тестовой выборке составило ROC AUC = 0.880
- Однако это модель очень нестабильна относительно признаков, на графике зависимости можно наблюдать сразу 4 пика, что говорит о возможном недообучении.

*Интерпретация важности признаков через shap*

In [ ]:
X_test_transformed = rfecv_tree.transform(X_test)
final_feature_names = np.array(rfecv_tree.get_feature_names_out())
explainer = shap.TreeExplainer(rfecv_tree.named_steps['model'].estimator_, feature_names=final_feature_names)
shap_values = explainer.shap_values(X_test_transformed)[:, :, 0]
shap.summary_plot(shap_values, X_test_transformed, feature_names=final_feature_names, plot_type='violin')

- Самой влиятельной фичой является `promotional_purchases`, который обратнопропорционален таргету (что вполне логично)
- Также влиятельными считаются признаки `pages_per_visit` (прямопропорционально) и `spent_time_prev_month` (прямопропорционально).
- Распределение вышеперечисленных признаков равномерно и вправо, и влево, а также достаточно амплитудно, что есть хорошо.

_____
### <a id='toc1_8_6_'></a>[KNN](#toc0_)

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_6_1_'></a>[Подбор гиперпараметров](#toc0_)

In [ ]:
pipeline = get_pipepline_clf(
    feature_selection='SelectKBest',
    numerical_columns=market.select_dtypes(include='number').columns,
    categorical_columns=['popular_category'],
    categorical_ordered_columns=['service_type', 'allow_notifications']
)

model_params_cat = {'model': [KNeighborsClassifier()],
                    'model__weights': ['uniform', 'distance']}
model_params_num = {
    'model__n_neighbors': lambda trial: trial.suggest_int('model__n_neighbors', 3, 100),
    'feature_selection__k': lambda trial: trial.suggest_int('feature_selection__k', 2, X_train.shape[1]),
}

study_knn = optuna.create_study(direction='maximize')

In [ ]:
%%time
study_knn.optimize(objective, n_trials=300, show_progress_bar=True)

In [ ]:
print(f'Best ROC AUC: {study_knn.best_value:.3f}')
best_params = get_best_params(study_knn.best_params)
print('\nBest parameters:')
pprint(best_params)

*Важность признаков судя по перебору*

In [ ]:
plot_param_importances(study_knn)

- Наиболее значимым гиперпараметром считается `feature_selection__k`
- Также стоит отметить вклад параметров `preprocessor__categorical_ordered__encoder`, `preprocessor__categorical_ordered__scaler` и `preprocessor__categorical__encoder`

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_6_2_'></a>[Feature Engineering](#toc0_)

*Обучим модель на лучших параметрах*

In [ ]:
knn = clone(pipeline).set_params(**best_params)
knn.fit(X_train, y_train.cat.codes)
knn

In [ ]:
feature_selection_perfomance(study_knn.trials_dataframe().groupby('params_feature_selection__k')['value'].max(),
                             model_names=['KNN'])

- ROC AUC на тестовой выборке составляет 0.923
- Модель показывает лучшие результаты на 8 признаках
- Можно заметить, что модель после пика идет по нисходящей, видимо из-за переобучения

_____
### <a id='toc1_8_7_'></a>[Метод опорных векторов](#toc0_)

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_7_1_'></a>[Подбор гиперпараметров](#toc0_)

In [ ]:
pipeline = get_pipepline_clf(
    feature_selection='SelectKBest',
    numerical_columns=market.select_dtypes(include='number').columns,
    categorical_columns=['popular_category'],
    categorical_ordered_columns=['service_type', 'allow_notifications']
)

model_params_cat = {'model': [SVC(random_state=RANDOM_STATE)],
                    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
model_params_num = {
    'model__C': lambda trial: trial.suggest_float('model__C', 1e-3, 1e3, log=True),
    'model__gamma': lambda trial: trial.suggest_float('model__gamma', 1e-3, 1, log=True),
    'model__degree': lambda trial: trial.suggest_int('model__degree', 2, 4),
    'model__coef0': lambda trial: trial.suggest_float('model__coef0', 0.0, 10.0),
    'feature_selection__k': lambda trial: trial.suggest_int('feature_selection__k', 2, X_train.shape[1]),
}

study_svc = optuna.create_study(direction='maximize')

In [ ]:
%%time
study_svc.optimize(objective, n_trials=300, show_progress_bar=True, timeout=900)

In [ ]:
print(f'Best ROC AUC: {study_svc.best_value:.3f}')
best_params = get_best_params(study_svc.best_params)
print('\nBest parameters:')
pprint(best_params)

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_8_7_2_'></a>[Feature Engineering](#toc0_)

*Важность признаков судя по перебору*

In [ ]:
plot_param_importances(study_svc)

- Наиболее значимыми признакоми считаются `model__gamma`, `feature_selection__k` и `preprocessor__categorical_ordered__encoder`
- Также стоит отметить важные признаки `preprocessor__categorical_ordered__scaler`, `preprocessor__categorical__encoder`,  `preprocessor__categorical_ordered__inputer_after`

*Обучим модель на лучших параметрах*

In [ ]:
svc = clone(pipeline).set_params(**best_params)
svc.fit(X_train, y_train.cat.codes)
svc

In [ ]:
feature_selection_perfomance(study_svc.trials_dataframe().groupby('params_feature_selection__k')['value'].max(),
                             model_names=['Метод опорных векторов'])

- Результаты топ модели на тестововой выборке составляет 0.908
- Лучший результат модель показывает на 12 фичах, после чего модель выходит на плато.

_____
### <a id='toc1_8_8_'></a>[Вывод](#toc0_)

In [ ]:
feature_selection_perfomance(rfecv_logreg.named_steps['model'].cv_results_,
                             rfecv_tree.named_steps['model'].cv_results_,
                             study_knn.trials_dataframe().groupby('params_feature_selection__k')['value'].max(),
                             study_svc.trials_dataframe().groupby('params_feature_selection__k')['value'].max(),
                             model_names=['Логистическая регрессия', 'Дерево решений', 'KNN', 'Метод опорных векторов'])

*Лучше всех на валидации показала себя модель KNN. Посмотрим ее качество на тестовой выборке*

In [ ]:
print(f'ROC AUC KNN на тестовой выборки = {roc_auc_score(y_test.cat.codes, knn.predict_proba(X_test)[:, 1]):.3f}')

- Хороший результат, возьмем эту модель как итоговую

In [ ]:
knn

*Отобранные моделью признаки*

In [ ]:
knn.named_steps['preprocessor'].get_feature_names_out()[knn.named_steps['feature_selection'].get_support()]

_____
### <a id='toc1_8_9_'></a>[Вывод:](#toc0_)

1. Логистическая регрессия
- Показала стабильные результаты.
- Лучший ROC AUC составил 0.899.
- Логистическая регрессия показала устойчивость к уменьшению числа признаков, сохраняя высокое качество при 8 признаках.
- Легко интерпретируемая модель, что является значимым фактором для бизнеса.

2. Дерево решений
- Продемонстрировало хороший результат (ROC AUC = 0.878) при использовании небольшого количества признаков (5).
- Однако очень нестабильно относительно кол-ва фичей.

3. K-Nearest Neighbors (KNN)
- Достиг максимального ROC AUC 0.923 (лучший результат) + аналогичный результат и на тестовой выборке (0.921).
- Модель легко интерпретируема, что важно для бизнеса.

4. Метод опорных векторов (SVC)
- Наивысший ROC AUC составил 0.907, однако показала менее нестабильное поведение при разном количестве признаков.
- Модель менее интерпретируема, а качество на малом числе признаков нестабильно, что усложняет применение в реальных условиях.


<hr style="border: none; border-top: 2px dashed;">

- Итоговая модель: KNN
    
    Выбор итоговой модели обусловлен следующими причинами:
    - Интерпретируемость: Позволяет легко понять значимость отдельных признаков и их влияние на целевую переменную, что важно для принятия решений в бизнесе.
    - Качество: Лучшее среди обученных моделей.

_____
_____
## <a id='toc1_9_'></a>[Сегментация покупателей](#toc0_)

Для чистоты анализа возьмем тестовую выборку для анализа. Добавим в нее предсказания выбранной модели. Также добавим к нашему датасету данные из таблицы `money`.

In [ ]:
y_test

- `Снизилась` = 1
- `Прежний уровень` = 0

In [ ]:
segmentation = X_test.copy()
segmentation = segmentation.merge(money, on='id')
segmentation['probability_of_decrease'] = knn.predict_proba(X_test)[:, 1]
segmentation['customer_activity'] = y_test.cat.codes
market_columns_translate.update({'Прибыль': 'profit', 'Вероятность снижения покупательской активности': 'probability_of_decrease'})
segmentation.head()

In [ ]:
statistical_graphis_for_numeric(segmentation.probability_of_decrease,
                                axis_title=market_columns_translate.inverse['probability_of_decrease'],
                                nbinsx=40)

- Видно, что распределение двумодельное. Второй горб описывает клиентов, склонных на снижение покупательской активности. Можно однозначно сказать, что есть явная причина, по которой часть клиентов уходят, приэтом другую часть покупателей это не задело. 

### <a id='toc1_9_1_'></a>[Категории для сегментации](#toc0_)

<img src="https://pictures.s3.yandex.net/resources/image_1695485033.png" width="1000"/>

In [ ]:
color_palette = cycle(px.colors.qualitative.Plotly)


def pieplot_grouped_by_customer_activity(data: pd.DataFrame, column, top_n_in_pie=5):
    """
    Функция для построения двух круговых диаграмм, показывающих распределение значений категориального столбца
    для двух групп данных: с уменьшенной активностью и с сохраненной активностью.

    Группировка данных происходит по значениям в колонке `customer_activity`, где:
    - `0` — означает снижение активности.
    - `1` — означает сохранение прежнего уровня активности.

    На каждой диаграмме отображаются топ-N категорий для выбранного столбца с категорией "Остальные" для оставшихся значений.

    Parameters
    ----------
    data : pd.DataFrame
        Входные данные в виде DataFrame, содержащие колонку `customer_activity` и категориальный столбец `column`, для которого строятся диаграммы.
    
    column : str
        Имя категориальной колонки, для которой строятся круговые диаграммы. Эта колонка должна быть частью DataFrame.
    
    top_n_in_pie : int, по умолчанию 5
        Количество топ-N категорий, которые будут отображены на каждой диаграмме. Остальные категории будут объединены в одну категорию "Остальные".
    """ 
    fig = make_subplots(
        rows=1, cols=2, specs=[[ {'type': 'domain'}, {'type': 'domain'}]],
        subplot_titles=('Уровень активности снизился', 'Прежний уровень активности')
    )
    
    category_agg = data[data.customer_activity == 0][column].value_counts()
    other = [category_agg[top_n_in_pie:].sum()]
    fig.add_trace(
        go.Pie(labels=category_agg.head(top_n_in_pie).index.tolist() + (['Остальные'] if other[0] else []),
               values=category_agg.head(top_n_in_pie).values.tolist() + (other if other[0] else []),
               name='',
               textinfo='label+percent'),
        row=1, col=1
    )
    
    category_agg = data[data.customer_activity == 1][column].value_counts()
    other = [category_agg[top_n_in_pie:].sum()]
    fig.add_trace(
        go.Pie(labels=category_agg.head(top_n_in_pie).index.tolist() + (['Остальные'] if other[0] else []),
               values=category_agg.head(top_n_in_pie).values.tolist() + (other if other[0] else []),
               name='',
               textinfo='label+percent'),
        row=1, col=2
    )

    fig.update_layout(
        title_text=f'Круговые диаграммы по колонке <b>{market_columns_translate.inverse[column]}</b><br>(<b>{column}</b>)',
        title_x=0.5,
        showlegend=True,
        width=1200,
        height=600,
    )

    fig.show()


def histogram_grouped_by_customer_activity(data: pd.DataFrame, column, nbinsx):
    """
    Функция для построения гистограмм, показывающих распределение значений числовой колонки
    для двух групп данных: с уменьшенной активностью и с сохраненной активностью.

    Группировка данных происходит по значениям в колонке `customer_activity`, где:
    - `0` — означает снижение активности.
    - `1` — означает сохранение прежнего уровня активности.

    Строятся две гистограммы, каждая для одной из групп с указанием частоты значений для выбранного столбца.

    Parameters
    ----------
    data : pd.DataFrame
        Входные данные в виде DataFrame, содержащие колонку `customer_activity` и числовой столбец `column`, для которого строятся гистограммы.
    
    column : str
        Имя числовой колонки, для которой строятся гистограммы. Эта колонка должна быть частью DataFrame.
    
    nbinsx : int
        Количество корзин (бин) для гистограммы.
    """
    data_class_0 = data[data.customer_activity == 0][column]
    data_class_1 = data[data.customer_activity == 1][column]

    fig = go.Figure()

    fig.add_trace(
        go.Histogram(
            x=data_class_0,
            name='Уровень активности снизился',
            opacity=0.5,
            marker=dict(color='blue'),
            nbinsx=nbinsx
        )
    )

    fig.add_trace(
        go.Histogram(
            x=data_class_1,
            name='Прежний уровень активности',
            opacity=0.5,
            marker=dict(color='orange'),
            nbinsx=nbinsx
        )
    )

    fig.update_layout(
        title_text=f'Гистограммы по колонке <b>{market_columns_translate.inverse[column]}</b><br>(<b>{column}</b>)',
        xaxis_title=column,
        yaxis_title='Частота',
        barmode='overlay',
        legend=dict(title="Класс активности"),
        width=900,
        height=500
    )

    fig.show()


def draw_scatter_between_probability_of_decrease_and(data: pd.DataFrame, num_col, cat_cols, opacity=0.4, rows=2, cols=2, height=1200, width=1200):
    """
    Функция для построения диаграмм рассеяния между вероятностью снижения активности 
    и числовым столбцом, разбитым по классам категориальных признаков.

    Для каждого категориального признака (из списка `cat_cols`) строится диаграмма рассеяния 
    для каждого уникального значения категории, а также линия тренда для каждой группы. 
    Также отображается общая линия тренда для всего датасета.

    Parameters
    ----------
    data : pd.DataFrame
        Входные данные в виде DataFrame, содержащие как минимум два столбца:
        - числовой столбец `num_col`, по которому строится график.
        - колонка с вероятностью снижения активности `probability_of_decrease`.
        - Набор категориальных признаков для группировки.
    
    num_col : str
        Имя числового столбца в DataFrame, для которого строится диаграмма рассеяния.

    cat_cols : list[str]
        Список категориальных признаков, по которым будет выполняться группировка данных 
        и строиться отдельная диаграмма рассеяния.

    opacity : float, по умолчанию 0.4
        Степень прозрачности точек на графике (от 0 до 1).

    rows : int, по умолчанию 2
        Количество строк для размещения графиков в сетке.

    cols : int, по умолчанию 2
        Количество столбцов для размещения графиков в сетке.

    height : int, по умолчанию 1200
        Высота итогового графика в пикселях.

    width : int, по умолчанию 1200
        Ширина итогового графика в пикселях.
    """
    fig = make_subplots(
        rows=rows, cols=cols, vertical_spacing=0.09,
        subplot_titles=[market_columns_translate.inverse[col] for col in data.select_dtypes(include='object').columns]
    )

    for cetegoty_i, category_col in enumerate(cat_cols):
        # Перебор категориальных признаков

        for class_of_category, group_of_category in data.groupby(category_col):
            # Перебор по каждому классу категории

            color = next(color_palette)
            x = group_of_category[num_col]
            y = group_of_category['probability_of_decrease']

            # диаграмма рассеяния по классу
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y,
                    mode='markers',
                    marker=dict(size=7, opacity=opacity, color=color),
                    name=class_of_category,
                    legendgroup=market_columns_translate.inverse[category_col],
                    legendgrouptitle=dict(text=market_columns_translate.inverse[category_col]),
                ),
                row=cetegoty_i // cols + 1, col=cetegoty_i % cols + 1
            )
            slope, intercept = np.polyfit(x, y, 1)
            trend_line = slope * x + intercept

            # Линия тренда по классу
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=trend_line,
                    mode='lines',
                    line=dict(
                        color=f'rgb{find_intermediate_color(hex_to_rgb(color), [0] * 3, 0.4)}',
                        width=3
                    ),
                    name='Линия тренда',
                    visible='legendonly',
                    legendgroup=market_columns_translate.inverse[category_col],
                ),
                row=cetegoty_i // cols + 1, col=cetegoty_i % cols + 1
            )

        # Общая линия тренда
        slope, intercept = np.polyfit(data[num_col], data.probability_of_decrease, 1)
        trend_line = slope * data[num_col] + intercept
        fig.add_trace(
            go.Scatter(
                x=data[num_col],
                y=trend_line,
                mode='lines',
                name='Общая линия тренда',
                visible='legendonly',
                line=dict(color='black', width=3),
                legendgroup=market_columns_translate.inverse[category_col],
            ),
            row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1
        )
        fig.update_xaxes(title_text=market_columns_translate.inverse[num_col], row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1)
        fig.update_yaxes(title_text=market_columns_translate.inverse['probability_of_decrease'], row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1)

    title=(f'Диаграммы рассеяния между признаками <b>Вероятность снижения активности</b><br>и <b>{market_columns_translate.inverse[num_col]}</b>,' +
           f'разбитые по классам категориальных признаков<br>corr={data.probability_of_decrease.corr(data[num_col]):.3f}')
    fig.update_layout(
        title=title,
        title_y=0.98,
        title_x=0.5,
        height=height,
        width=width,
        legend=dict(
            traceorder='grouped',
            groupclick='toggleitem',
        ),
    )
    fig.show()

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_9_1_1_'></a>[Коммуникация с клиентом](#toc0_)

In [ ]:
communication  = segmentation[['service_type', 'allow_notifications', 'marketing_activity_6_months', 'current_month_marketing_activity', 'registration_duration', 'probability_of_decrease', 'customer_activity']]
communication.head()

In [ ]:
cat_cols = ['service_type', 'allow_notifications', 'current_month_marketing_activity']
for cat_col in ['service_type', 'allow_notifications', 'current_month_marketing_activity']:
    pieplot_grouped_by_customer_activity(communication, cat_col)

- По круговой диаграмме `Тип сервиса` видно, что покупатальская активность снизилась больше у клиентов с типом сервиса премиум 

In [ ]:
for num_col in ['marketing_activity_6_months', 'registration_duration']:
    histogram_grouped_by_customer_activity(communication, num_col, nbinsx=20)
    draw_scatter_between_probability_of_decrease_and(communication, num_col, cat_cols, rows=2, height=900)

- По гистограмме `Маркет_актив_6_мес` видно, что клиенты, которым было уделено меньше маркетингого привлечения, имеет спад поупательской активность (все логично).

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_9_1_2_'></a>[Продуктовое поведение](#toc0_)

In [ ]:
product_behavior = segmentation[['popular_category', 'avg_categories_per_visit', 'unpaid_products_quarter', 'probability_of_decrease', 'customer_activity']]
product_behavior.head()

In [ ]:
for cat_col in ['popular_category', 'unpaid_products_quarter']:
    pieplot_grouped_by_customer_activity(product_behavior, cat_col, top_n_in_pie=7)

- Видно, что из-за уменьшения продаж "Техника для красоты и здоровья" и "Мелкая бытовая техника и электроника" уменьшилась и покупательская активность. Вполне возможно что уменьшение продаж этих категорий напрямую связано с уменьшением поставок, следственно некоторые клиенты купили себе товары этой категории в других магазинах.

In [ ]:
histogram_grouped_by_customer_activity(product_behavior, 'avg_categories_per_visit', nbinsx=20)
draw_scatter_between_probability_of_decrease_and(product_behavior, 'avg_categories_per_visit', ['popular_category', 'unpaid_products_quarter'], rows=1, height=650)

- Гистограмма подтверждает предыдущий вывод о нехватке ассортимента на полках магазина, так как видно, что клиент просмматривает меньшее кол-во категорий, из-за нехватки продукции.

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_9_1_3_'></a>[Поведение на сайте](#toc0_)

In [ ]:
site_behavior = segmentation[
    ['spent_time_cur_month', 'spent_time_prev_month', 'pages_per_visit', 'service_errors', 'probability_of_decrease', 'customer_activity']
].merge((market_time.groupby('id')['minutes'].sum() / 2), on='id').rename(columns={'minutes': 'avg_minutes'})
site_behavior.head()

In [ ]:
for num_col in ['spent_time_cur_month', 'spent_time_prev_month', 'pages_per_visit', 'service_errors']:
    histogram_grouped_by_customer_activity(site_behavior, num_col, nbinsx=20)
    draw_scatter_between_probability_of_decrease_and(site_behavior, num_col, ['customer_activity'], opacity=0.7, rows=1, cols=1, height=650, width=900)

- На данных гистограммах можно отчетливо увидеть, что время, проведенное на сайте упало у клиентов с низким уровнем активности. С одной стороны это поледствие уменьшение активности, однако такую природу гистограмм можно описать и с другой стороны, к примеру: Пользователь не нашел нужных товаров, поэтому пошел искать в другой магазин.

<hr style="border: none; border-top: 3px dashed;">

#### <a id='toc1_9_1_4_'></a>[Финансовое поведение](#toc0_)

In [ ]:
financial_behavior = segmentation[['revenue_cur_month', 'probability_of_decrease', 'customer_activity']]
financial_behavior['revenue_diff_cur_prev_month'] = X_test['revenue_cur_month'] - X_test['revenue_prev_month']
financial_behavior['revenue_diff_prev_preprev_month'] = X_test['revenue_prev_month'] - X_test['revenue_preprev_month']
financial_behavior.head()

In [ ]:
market_columns_translate.update({'Разница в выручке между текущим и прошлым месяцами': 'revenue_diff_cur_prev_month', 'Разница в выручке между прошлым и позапрошлым месяцями': 'revenue_diff_prev_preprev_month'})

In [ ]:
for num_col in ['revenue_cur_month', 'revenue_diff_cur_prev_month', 'revenue_diff_prev_preprev_month']:
    histogram_grouped_by_customer_activity(financial_behavior, num_col, nbinsx=40)
    draw_scatter_between_probability_of_decrease_and(financial_behavior, num_col, ['customer_activity'], opacity=0.7, rows=1, cols=1, height=650, width=900)

- Здесь мы можем наблюдать уменьшение выручки с клиентов, покупательская активность которых падает, это не причина, а следствие.

_____
### <a id='toc1_9_2_'></a>[Вывод](#toc0_)

1. **Сегментация покупателей**:
    - Распределение вероятности снижения покупательской активности имеет двумодальное распределение, что указывает на наличие двух различных групп клиентов.
    - Клиенты с типом сервиса "премиум" имеют более высокую вероятность снижения покупательской активности по сравнению с клиентами с типом сервиса "стандарт".
    - Клиенты, которым было уделено меньше маркетингового внимания за последние 6 месяцев, имеют более высокую вероятность снижения покупательской активности.

2. **Продуктовое поведение**:
    - Снижение покупательской активности связано с уменьшением продаж в категориях "Техника для красоты и здоровья" и "Мелкая бытовая техника и электроника".
    - Клиенты, которые просматривают меньшее количество категорий за визит, имеют более высокую вероятность снижения покупательской активности, что может указывать на нехватку ассортимента.

3. **Поведение на сайте**:
    - Клиенты с низким уровнем активности проводят меньше времени на сайте, что может быть как следствием, так и причиной снижения покупательской активности.
    - У клиентов с низким уровнем активности также наблюдается меньшее количество страниц за визит и большее количество ошибок сервиса.

4. **Финансовое поведение**:
    - Снижение покупательской активности сопровождается уменьшением выручки от клиентов, что является следствием снижения их активности.
    - Клиенты с высокой вероятностью снижения покупательской активности показывают отрицательную динамику выручки за последние месяцы.

- ***Предложения бизнесу***
1. **Увеличение маркетинговой активности**:
    - **Причина**: Анализ показал, что клиенты с низкой маркетинговой активностью за последние 6 месяцев имеют более высокую вероятность снижения покупательской активности.
    - **Предложение**: Увеличить количество маркетинговых кампаний, направленных на этот сегмент клиентов, с акцентом на персонализированные предложения и скидки.

2. **Оптимизация ассортимента товаров**:
    - **Причина**: Клиенты, которые просматривают меньшее количество категорий товаров, имеют более высокую вероятность снижения покупательской активности. Это может быть связано с нехваткой интересующих их товаров.
    - **Предложение**: Провести анализ ассортимента и увеличить наличие популярных категорий товаров, таких как "Техника для красоты и здоровья" и "Мелкая бытовая техника и электроника".

3. **Улучшение пользовательского опыта на сайте**:
    - **Причина**: Клиенты, которые проводят меньше времени на сайте и сталкиваются с ошибками сервиса, имеют более высокую вероятность снижения покупательской активности.
    - **Предложение**: Оптимизировать работу сайта, уменьшить количество ошибок сервиса и улучшить навигацию, чтобы пользователи могли легко находить интересующие их товары.

4. **Персонализированные уведомления**:
    - **Причина**: Клиенты, которые разрешили получать уведомления, имеют более высокую вероятность снижения покупательской активности.
    - **Предложение**: Персонализировать уведомления, чтобы они были более релевантными и интересными для клиентов, что может повысить их вовлеченность и активность.

_____
_____
## <a id='toc1_10_'></a>[Общий вывод](#toc0_)

**Модели**
1. **Логистическая регрессия**:
    - Показала стабильные результаты.
    - Лучший ROC AUC составил 0.899.
    - Логистическая регрессия показала устойчивость к уменьшению числа признаков, сохраняя высокое качество при 8 признаках.
    - Легко интерпретируемая модель, что является значимым фактором для бизнеса.
2. **Дерево решений**:
    - Продемонстрировало хороший результат (ROC AUC = 0.881) при использовании большого количества признаков (11).
    - Однако уступает другим моделям в стабильности качества.

3. **K-Nearest Neighbors (KNN)**:
    - Достиг максимального ROC AUC 0.923 (лучший результат)

4. **Метод опорных векторов (SVC)**:
    - Наивысший ROC AUC составил 0.910.
    - Модель менее интерпретируема.

- **KNN** была выбрана в качестве итоговой модели по следующим причинам:
    - **Интерпретируемость**: позволяет легко понять значимость отдельных признаков и их влияние на целевую переменную, что важно для принятия решений в бизнесе.
    - **Стабильность**: Модель высокое качество независимо от числа используемых признаков, что делает ее надежным выбором.
    - **Качество**: С отрывом лучше остальных.

**Предложения бизнесу**
1. **Увеличение маркетинговой активности**:
    - **Причина**: Анализ показал, что клиенты с низкой маркетинговой активностью за последние 6 месяцев имеют более высокую вероятность снижения покупательской активности.
    - **Предложение**: Увеличить количество маркетинговых кампаний, направленных на этот сегмент клиентов, с акцентом на персонализированные предложения и скидки.
2. **Оптимизация ассортимента товаров**:
    - **Причина**: Клиенты, которые просматривают меньшее количество категорий товаров, имеют более высокую вероятность снижения покупательской активности. Это может быть связано с нехваткой интересующих их товаров.
    - **Предложение**: Провести анализ ассортимента и увеличить наличие популярных категорий товаров, таких как "Техника для красоты и здоровья" и "Мелкая бытовая техника и электроника".
3. **Улучшение пользовательского опыта на сайте**:
    - **Причина**: Клиенты, которые проводят меньше времени на сайте и сталкиваются с ошибками сервиса, имеют более высокую вероятность снижения покупательской активности.
    - **Предложение**: Оптимизировать работу сайта, уменьшить количество ошибок сервиса и улучшить навигацию, чтобы пользователи могли легко находить интересующие их товары.
4. **Персонализированные уведомления**:
    - **Причина**: Клиенты, которые разрешили получать уведомления, имеют более высокую вероятность снижения покупательской активности.
    - **Предложение**: Персонализировать уведомления, чтобы они были более релевантными и интересными для клиентов, что может повысить их вовлеченность и активность.